<a href="https://colab.research.google.com/github/srnarasim/TAOExperiment/blob/main/TAOExperiment_fixed_updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TAO Experiment - Text Classification with Test-time Adaptation

In [17]:
# Create a CSV file with sample content

import csv

data = [
    ['Product', 'Product Description', 'Category'],
    ['Wireless Bluetooth headphones with noise cancellation', 'Headphones', 'Electronics'],
    ['Smartphone with OLED display and 128GB storage', 'Smartphone', 'Electronics'],
    ['Gaming laptop with high refresh rate screen', 'Laptop', 'Electronics'],
    ['Smart home security camera with night vision', 'Smart Home Device', 'Electronics'],
    ['Cotton t-shirt with graphic print design', 'T-shirt', 'Clothing'],
    ['Wooden dining table with six matching chairs', 'Dining Table', 'Furniture'],
    ['Genuine leather wallet with multiple card slots', 'Wallet', 'Accessories'],
    ['Insulated stainless steel water bottle', 'Water Bottle', 'Kitchen']
]

with open('balanced_data.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(data)

!cat balanced_data.csv

Product,Product Description,Category
Wireless Bluetooth headphones with noise cancellation,Headphones,Electronics
Smartphone with OLED display and 128GB storage,Smartphone,Electronics
Gaming laptop with high refresh rate screen,Laptop,Electronics
Smart home security camera with night vision,Smart Home Device,Electronics
Cotton t-shirt with graphic print design,T-shirt,Clothing
Wooden dining table with six matching chairs,Dining Table,Furniture
Genuine leather wallet with multiple card slots,Wallet,Accessories
Insulated stainless steel water bottle,Water Bottle,Kitchen


In [18]:
!pip install datasets transformers accelerate bitsandbytes

In [19]:
import pandas as pd
from transformers import AutoTokenizer

# Load dataset
df = pd.read_csv("balanced_data.csv")

# Load tokenizer (using BERT model)
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Create category mapping
category_mapping = {category: idx for idx, category in enumerate(df["Category"].unique())}
df["Label"] = df["Category"].map(category_mapping)

# Tokenize product descriptions
max_length = 128
encoded_data = tokenizer(
    df["Product Description"].tolist(),
    padding=True,
    truncation=True,
    max_length=max_length,
    return_tensors="pt"
)

# Store tokenized data in DataFrame
df["input_ids"] = encoded_data["input_ids"].tolist()
df["attention_mask"] = encoded_data["attention_mask"].tolist()

print("Tokenization completed. DataFrame columns:", df.columns.tolist())

Tokenization completed. DataFrame columns: ['Product', 'Product Description', 'Category', 'Label', 'input_ids', 'attention_mask']


In [20]:
import torch
from datasets import Dataset
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments, default_data_collator

# ... (rest of your code) ...

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",  # Ensure this matches the metric returned by compute_metrics
    # ... (rest of your training arguments) ...
)

# Define compute_metrics function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    # Calculate and return the accuracy
    return {"accuracy": (preds == labels).mean()}

# Initialize trainer, include compute_metrics
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_test["train"],
    eval_dataset=train_test["test"],
    compute_metrics=compute_metrics # Pass the function to the Trainer
)

# Train the model
print("Starting training...")
trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Starting training...


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.761240,0.500000
2,No log,1.787304,0.500000
3,No log,1.665977,0.500000
4,No log,1.784607,0.500000
5,No log,1.784536,0.500000


TrainOutput(global_step=10, training_loss=1.2079913139343261, metrics={'train_runtime': 80.5321, 'train_samples_per_second': 0.373, 'train_steps_per_second': 0.124, 'total_flos': 77085393300.0, 'train_loss': 1.2079913139343261, 'epoch': 5.0})

## Test-time Adaptation for New Categories
This section implements test-time adaptation to detect products from new, unseen categories.

In [21]:
import torch.nn.functional as F
import numpy as np

def predict_product_category(text, entropy_threshold=1.5, confidence_threshold=0.4):
    # Tokenize input
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=128)

    # Get model predictions
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

        # Convert to probabilities using softmax
        probabilities = F.softmax(logits, dim=-1).cpu().numpy()[0]

        # Calculate entropy
        entropy = -np.sum(probabilities * np.log(probabilities + 1e-9))

        # Calculate confidence
        confidence = np.max(probabilities)

        # Make prediction
        if entropy > entropy_threshold or confidence < confidence_threshold:
            print(f"Product '{text}' might be a NEW category!")
            print(f"Entropy: {entropy:.3f}, Confidence: {confidence:.3f}")
            return 'New Category'
        else:
            predicted_idx = np.argmax(probabilities)
            category = list(category_mapping.keys())[predicted_idx]
            print(f"Product '{text}' classified as: {category}")
            print(f"Confidence: {confidence:.3f}, Entropy: {entropy:.3f}")
            return category

# Test with various product descriptions
test_products = [
    # Known categories
    "Wireless gaming headphones with RGB lighting",
    "Wooden dining table with extendable leaf",
    "Classic leather wallet with coin pocket",
    "Cotton polo shirt with embroidered logo",

    # Potentially new categories
    "Smart fitness tracker with heart rate monitor",
    "Electric scooter with foldable design",
    "Organic green tea from Japan",
    "Professional oil painting set with easel",
    "Garden tools set with pruning shears",
    "Yoga mat with alignment lines"
]

print("Testing product classification with test-time adaptation:\n")
results = []
for product in test_products:
    category = predict_product_category(product)
    results.append({'Product': product, 'Predicted Category': category})
    print("-" * 80 + "\n")

# Display results in a DataFrame
results_df = pd.DataFrame(results)
print("\nSummary of Results:")
print(results_df)

Testing product classification with test-time adaptation:

Product 'Wireless gaming headphones with RGB lighting' might be a NEW category!
Entropy: 1.531, Confidence: 0.285
--------------------------------------------------------------------------------

Product 'Wooden dining table with extendable leaf' might be a NEW category!
Entropy: 1.530, Confidence: 0.288
--------------------------------------------------------------------------------

Product 'Classic leather wallet with coin pocket' might be a NEW category!
Entropy: 1.548, Confidence: 0.274
--------------------------------------------------------------------------------

Product 'Cotton polo shirt with embroidered logo' might be a NEW category!
Entropy: 1.543, Confidence: 0.292
--------------------------------------------------------------------------------

Product 'Smart fitness tracker with heart rate monitor' might be a NEW category!
Entropy: 1.532, Confidence: 0.285
--------------------------------------------------------

In [22]:
# Create a proper dataset from the tokenized data
from datasets import Dataset

# Convert lists stored as strings back to actual lists if needed
import ast

# Create a function to safely convert string representations of lists to actual lists
def safe_eval(x):
    if isinstance(x, list):
        return x
    try:
        return ast.literal_eval(x)
    except (ValueError, SyntaxError):
        return x

# Prepare the dataset
dataset_dict = {
    'input_ids': [safe_eval(ids) for ids in df['input_ids']],
    'attention_mask': [safe_eval(mask) for mask in df['attention_mask']],
    'labels': df['Label'].tolist()
}

# Create the dataset
new_dataset = Dataset.from_dict(dataset_dict)
print(f"Created dataset with {len(new_dataset)} examples")
print(f"Dataset features: {new_dataset.features}")


Created dataset with 8 examples
Dataset features: {'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None), 'labels': Value(dtype='int64', id=None)}


In [23]:
from peft import get_peft_model, LoraConfig
from transformers import default_data_collator, Trainer, TrainingArguments

# ... (rest of your code) ...

# Reinitialize trainer with updated training_args and data_collator, pass compute_metrics to the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=new_dataset,  # Use new_dataset for fine-tuning
    eval_dataset=new_dataset,   # Use new_dataset for evaluation as well
    data_collator=default_data_collator,  # Ensure proper data collation
    compute_metrics=lambda pred: {'accuracy': (pred.predictions.argmax(-1) == pred.label_ids).mean()} # Pass compute_metrics here
)

# Modify the compute_loss method to accept num_items_in_batch
def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None): # Add num_items_in_batch as an argument
    """
    How the loss is computed by Trainer. By default, all models return the loss in the first element.
    Subclass and override for custom behavior.
    """
    if self.label_smoother is not None and "labels" in inputs:
        labels = inputs.pop("labels")
    else:
        labels = None

    # Remove num_items_in_batch from inputs if it exists to avoid issues with the model
    inputs.pop("num_items_in_batch", None)

    outputs = model(**inputs)

    # Save past state if it exists
    # TODO: this needs to be fixed and made cleaner later.
    if self.args.past_index >= 0:
        self._past_state = outputs[self.args.past_index]

    if labels is not None:
        loss = self.label_smoother(outputs, labels)
    else:
        # We don't use .loss here since the model may return tuples instead of ModelOutput.
        loss = outputs["loss"] if isinstance(outputs, dict) else outputs[0]

    return (loss, outputs) if return_outputs else loss

# Assign the modified compute_loss method to the trainer
trainer.compute_loss = compute_loss.__get__(trainer) # type: ignore

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.313603,0.500000
2,No log,1.239264,0.500000
3,No log,1.131488,0.625000
4,No log,1.065731,0.625000
5,No log,1.040174,0.625000


TrainOutput(global_step=10, training_loss=1.2382075309753418, metrics={'train_runtime': 63.5946, 'train_samples_per_second': 0.629, 'train_steps_per_second': 0.157, 'total_flos': 102780524400.0, 'train_loss': 1.2382075309753418, 'epoch': 5.0})

In [24]:
import torch.nn.functional as F
import numpy as np

def predict_product_category(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)

    probabilities = F.softmax(outputs.logits, dim=-1).cpu().numpy()[0]
    entropy = -np.sum(probabilities * np.log(probabilities + 1e-9))  # Compute entropy

    if entropy > 1.5:  # Threshold for uncertainty
        print(f"Product '{text}' is a potential NEW category!")
        return "New Category"
    else:
        predicted_label = np.argmax(probabilities)
        category = list(category_mapping.keys())[predicted_label]
        return category

# Example: New product descriptions
new_products = [
    "Smart fitness tracker with heart rate monitor",
    "Electric scooter with foldable design"
]

for product in new_products:
    print(predict_product_category(product))


Product 'Smart fitness tracker with heart rate monitor' is a potential NEW category!
New Category
Product 'Electric scooter with foldable design' is a potential NEW category!
New Category


In [25]:
from peft import get_peft_model, LoraConfig

# LoRA configuration
lora_config = LoraConfig(r=8, lora_alpha=16, lora_dropout=0.1)
model = get_peft_model(model, lora_config)

# Fine-tune only on new categories
new_product_data = [
    {"text": "Smart fitness tracker with heart rate monitor", "label": len(category_mapping)},
    {"text": "Electric scooter with foldable design", "label": len(category_mapping) + 1}
]
new_df = pd.DataFrame(new_product_data)
new_dataset = Dataset.from_pandas(new_df)

trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.131488,0.625000
2,No log,1.131488,0.625000
3,No log,1.131488,0.625000
4,No log,1.131488,0.625000
5,No log,1.131488,0.625000


TrainOutput(global_step=10, training_loss=1.1479059219360352, metrics={'train_runtime': 48.5092, 'train_samples_per_second': 0.825, 'train_steps_per_second': 0.206, 'total_flos': 103134418800.0, 'train_loss': 1.1479059219360352, 'epoch': 5.0})